# Main model training

In [1]:
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
display(HTML("<style>.container { width:99% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [90]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
drop_columns = ['image_id', 'patient_id']+[c for c in df_train.columns if c not in df_test.columns and c!='cancer']
df_train = df_train.drop(columns=drop_columns)

In [91]:
df_train.sample(10)

,site_id,laterality,view,age,cancer,implant,machine_id
30878,1,R,CC,67.0,0,0,49
14076,2,L,CC,54.0,0,0,21
48206,2,R,MLO,66.0,0,0,48
26796,1,R,MLO,44.0,0,0,49
2917,1,R,MLO,58.0,1,0,49
28585,2,R,CC,67.0,1,0,48
29733,2,R,MLO,54.0,0,0,21
23388,1,L,CC,61.0,1,0,49
44273,1,L,CC,51.0,0,0,49
92,1,R,CC,71.0,0,0,49


In [92]:
for c in df_train.columns:
    if c!='age':
        print(c)
        print(df_train[c].value_counts())
        print()

site_id
1    29519
2    25187
Name: site_id, dtype: int64

laterality
R    27439
L    27267
Name: laterality, dtype: int64

view
MLO    27903
CC     26765
AT        19
LM        10
ML         8
LMO        1
Name: view, dtype: int64

cancer
0    53548
1     1158
Name: cancer, dtype: int64

implant
0    53229
1     1477
Name: implant, dtype: int64

machine_id
49     23529
48      8699
29      8267
21      8221
93      1915
216     1908
210     1070
170      923
190      145
197       29
Name: machine_id, dtype: int64



In [95]:
categorical = ['site_id','laterality','view','implant','machine_id']
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
onehots = {c:OneHotEncoder() for c in categorical}
import numpy as np
for c in categorical:    
    transform = onehots[c].fit_transform(df_train[c].values[:].reshape(-1, 1)).toarray()
    c_name = [f'{c}_{i}' for i in range(transform.shape[1])]
    df_train[c_name] = transform
df_train = df_train.drop(columns=categorical)

In [101]:
!pip install xgboost
from xgboost import XGBClassifier
classifier = XGBClassifier()
X_train,y_train = df_train.drop(columns=['cancer']), df_train['cancer']
classifier.fit(X_train,y_train)

In [112]:
## Test
X_test = df_test[categorical+['age']]

for c in categorical:
    transform = onehots[c].transform(X_test[c].values[:].reshape(-1, 1)).toarray()
    c_name = [f'{c}_{i}' for i in range(transform.shape[1])]
    X_test[c_name] = transform
X_test = X_test.drop(columns=categorical)

predictions = classifier.predict(X_test)
df_test['cancer'] = predictions
submission = df_test[['prediction_id', 'cancer']]
submission.to_csv('sample_submission.csv')